Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Create Image
In this notebook, we show the following steps for deploying a web service using AzureML:
- Create an image
- Test image locally

In [ ]:
from azure_utils.machine_learning.realtime.image import get_or_create_image, lightgbm_test_image_locally, get_model
from notebooks import directory

AML will use the following information to create an image, provision a cluster and deploy a service. Replace the 
values in the following cell with your information.

## Create an image
We will now modify the `score.py` created in the previous notebook for the `init()` function to use the model we 
registered to the workspace earlier.

In [ ]:
%%writefile score.py

import sys
import json
import logging
import timeit as t
from azureml.core.model import Model
from azureml.contrib.services.aml_request import rawhttp
from azureml.contrib.services.aml_response import AMLResponse
from azure_utils.machine_learning.duplicate_model import DuplicateModel

sys.path.append('./scripts/')


def init():
    logger = logging.getLogger("scoring_script")
    global model
    model_name = 'question_match_model'
    model_path = Model.get_model_path(model_name)
    questions_path = './notebooks/data_folder/questions.tsv'
    start = t.default_timer()
    model = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    load_time_msg = "Model loading time: {0} ms".format(
        round((end - start) * 1000, 2))
    logger.info(load_time_msg)


@rawhttp
def run(request):
    """
    Function runs on each request
    """
    body = request.data
    if request.method == 'POST':
        logger = logging.getLogger("scoring_script")
        json_load_text = json.loads(body)
        text_to_score = json_load_text['input']
        start = t.default_timer()
        resp = model.score(text_to_score)
        end = t.default_timer()
        logger.info("Prediction took {0} ms".format(
            round((end - start) * 1000, 2)))
        return json.dumps(resp)
    if request.method == 'GET':
        resp_body = {
            "azEnvironment": "Azure",
            "location": "westus2",
            "osType": "Ubuntu 16.04",
            "resourceGroupName": "",
            "resourceId": "",
            "sku": "",
            "subscriptionId": "",
            "uniqueId": "PythonMLRST",
            "vmSize": "",
            "zone": "",
            "isServer": False,
            "version": ""
        }
        return resp_body
    return AMLResponse("bad request", 500)

Let's specify the conda and pip dependencies for the image.

In [ ]:
dependencies = ["./notebooks/data_folder/questions.tsv"]
models = [get_model(model_name='question_match_model')]

image = get_or_create_image(models=models, dependencies=dependencies)

## Test image locally

Now, let's use one of the duplicate questions to test our image.

In [ ]:
lightgbm_test_image_locally(image, directory)

## Conclusion

We can now move on to [Deploy on Azure Kubernetes Service](05_DeployOnAKS.ipynb) notebook. 